In [37]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'hw3'))
	print(os.getcwd())
except:
	pass

# Homework 3 - Renjie Zhu - A53266114

In [38]:
import random
import numpy as np

In [39]:
import gzip
from collections import defaultdict

def readGz(path):
  for l in gzip.open(path, 'rt'):
    yield eval(l)

def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')

def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [40]:
bookCount = defaultdict(int)
bookSet = set()
totalRead = 0
user_read = defaultdict(set)
data = []

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    user_read[user].add(book)
    bookSet.add(book)
    totalRead += 1
    data.append((user, book))

## 1.



In [41]:
training = data[:190000]
validation = data[190000:]
validation_new = []

In [42]:
for user, book in validation:
    sample = random.sample(bookSet.difference(user_read[user]), 1)[0]
    validation_new.append([user, book, 1])
    validation_new.append([user, sample, 0])
validation = np.array(validation_new)

In [43]:
# baseline model
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

popular = set()
count = 0
for ic, i in mostPopular:
    count += ic
    popular.add(i)
    if count > totalRead/2: break

In [44]:
pred = []
for _, book, _ in validation:
    if book in popular:
        pred.append(1)
    else:
        pred.append(0)


In [45]:
valid_gt = validation[:,2].astype(int)

In [46]:
print(f"Baseline accuracy: {sum(pred==valid_gt)/len(pred)}.")

Baseline accuracy: 0.6511.


## 2.

Better threshold

In [50]:
# better threshold
popular = set()
count = 0
for ic, i in mostPopular:
    count += ic
    popular.add(i)
    if count > totalRead/: break

In [51]:
pred = []
for _, book, _ in validation:
    if book in popular:
        pred.append(1)
    else:
        pred.append(0)

In [52]:
print(f"Baseline accuracy: {sum(pred==valid_gt)/len(pred)}.")

Baseline accuracy: 0.5944.


## 3.

